In [1]:
## Import packages

#Kspice
import sys
sys.path.append(r"C:\Program Files (x86)\Kongsberg\K-Spice\bin64") #add to path to allow kspice import
import kspice # if import error, check correct python version (3.11)

#Basic functionality
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#ML
import torch 

In [2]:
#Initialize simulator

project_path = r"C:\Appl\K-Spice-Projects\Kristin_v23" #Specify path to downloaded project.
_ = kspice.Simulator(project_path) #Create instance of project

In [3]:
#Disable power and wells - not API functionality

In [4]:
timeline = _.activate_timeline("Engineering") # Select the avaliable engineering timeline
app = "Topside" # We only make changes to the topside module NOTE: From software we can #deactivate Wells and Power in ESS model, can this be done from python? (If it increases speed)
timeline.initialize() #

In [5]:
#Load Kristin model, parameters and initial conditions

timeline.load_model("KristinMaria_master_disabledPowerWells") #Load model
timeline.load_parameters("KristinMaria_master_disabledPowerWells") # load fixed parameters
timeline.load_initial_condition("KristinMaria_master_disabledPowerWells") # Load initial conditions

In [8]:
#import sys
#del sys.modules['enviroment'] #to delete package memory - loading updates of Sim class
from enviroment import Sim

env = Sim(timeline, app) # Create env instance
env.import_variables("xlsx/xl_tester1.xlsx") #Import variables of interest

In [9]:
env.check_df("Global KPI") #Check random column in dataframe. i.e. correct labeling in correspondance to simulator

S27FQIC4011:Value ✓
S23KA001c:Power ✓
S23KA002c:Power ✓
S23KA003c:Power ✓


In [10]:
env.timeline.set_speed(10) #Set speed of simulator, this value is what we wish for not necesarly what we get
env.timeline.run()

In [11]:
env.timeline.achieved_speed #Here you can see actual speed

4.7252615590261415

In [12]:
state = env.reset() # We reset the process simulator by pausing it and again fetching the initial conditions

In [13]:
#For the future we want the state at a specific form 
import itertools
state_flat = np.array(list(itertools.chain.from_iterable(state)))
state_tensor = torch.tensor(state_flat, dtype=torch.float32) # make torch tensor

In [14]:
steps_done = 0
action = torch.tensor(env.sample("action"), dtype=torch.float32) # select action (up, down, stay) for each controller by a factor of 1.
print(action, action.shape)

tensor([1., 1., 1., 2., 0., 0., 2., 0., 2., 0.]) torch.Size([10])


In [ ]:
observation, reward, terminated, truncated = env.step(action) # Add changes to model and use a polling function to check for updates, timeout currently as 5 min to reach new setpoint with arg: 0.1 precision

S23LT1006 has reached its setpoint 44.97753579549194 [%], prog.list: [0, 2, 3, 4, 5, 6, 7, 8, 9] 

S23LT1066 has reached its setpoint 42.173556574557665 [%], prog.list: [0, 2, 3, 4, 6, 7, 8, 9] 

S23FIT1014 has reached its setpoint 12.995376914018834 [m3/h], prog.list: [0, 2, 3, 4, 7, 8, 9] 

S23FIT1107 has reached its setpoint 30.010019343898996 [m3/h], prog.list: [0, 2, 3, 4, 7, 9] 

S23TT1004 has reached its setpoint 29.98548261528657 [C], prog.list: [2, 3, 4, 7, 9] 

S23TT1034 has reached its setpoint 29.994326102701848 [C], prog.list: [3, 4, 7, 9] 



In [56]:
env.timeline.pause() # Pause simulator